In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import os
import numpy as np
import linecache #fast access to a specific file line
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from einops import rearrange, repeat
import torch.nn.functional as F
import time
from pathlib import Path

In [2]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

1.11.0
11.1
True


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
ALPHABET = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I",
            "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]

ALPHABET = {ALPHABET[i]:i for i in range(len(ALPHABET))}

ALPHABET['-']= 20
ALPHABET['Z']= 21

print(ALPHABET)

{'A': 0, 'R': 1, 'N': 2, 'D': 3, 'C': 4, 'Q': 5, 'E': 6, 'G': 7, 'H': 8, 'I': 9, 'L': 10, 'K': 11, 'M': 12, 'F': 13, 'P': 14, 'S': 15, 'T': 16, 'W': 17, 'Y': 18, 'V': 19, '-': 20, 'Z': 21}


In [5]:
class MyDataset(Dataset):
    def __init__(self, data_dir, cont_size=6,div=1400000,verbose=False):
        
        self.col_size = 60 #number of column per file (Fasta standard)
        self.data_dir = data_dir #directory of the dataset
        self.cont_size = cont_size
        self.div = div
        self.len = 0  #number of families of sequences (1 per file)
        self.paths = {} #path of each families in the folder
        self.seq_lens = {} #length of each member of the family
        self.seq_nums = {} #number of member of the family
        self.aa_freqs = {} #frequencies of each symbol in the sequence family
        
        
        dir_path = data_dir
        count = 0
        
        # Iterate directory
        for path in os.listdir(dir_path):
            # check if current path is a file
            temp_path = os.path.join(dir_path, path)
            if os.path.isfile(temp_path):
                n = 0 #number of sequences
                p = 0 # used to calculate the length of the sequences
                r = 0 # also used this way

                l = 0 # length of the seq l = p * self.col_size + r 

                cpt = 0 # to detect inconsistencies
                
                with open(temp_path, newline='') as f:
                    first_prot = True
                    newf = True
                    
                    aa_freq = torch.zeros(20)
                    
                    #parsing the file
                    line = f.readline()[:-1]
                    while line:
                        cpt += 1
                        if line[0] == '>': #header line
                            n += 1
                            if newf and not first_prot:
                                newf = False
                            first_prot = False
                                
                        else:# sequence line
                            if newf and len(line) == self.col_size:
                                p += 1

                            if newf and len(line) != self.col_size:
                                r = len(line)
                            for aa in line:
                                aa_id = ALPHABET.get(aa,21)
                                if aa_id < 20:
                                    aa_freq[aa_id] += 1



                            assert len(line) == self.col_size or len(line) == r
                        line = f.readline()[:-1]
                    
                    aa_freq = F.normalize(aa_freq,dim=0,p=1)

                l = p*self.col_size + r
                
                #sanity check
                #if the file line count is coherent with the number of sequences and their line count
                try: #if r != 0
                    assert (p+2) * n == cpt
                except: #if r == 0
                    assert (p+1) * n == cpt
                    assert r == 0
                    
                
                if n>1: #if this is false, we can't find pairs
                    self.paths[count] = path
                    self.seq_lens[count] = l
                    self.seq_nums[count] = n
                    self.aa_freqs[count] = aa_freq
                    count += 1
                    
                    if verbose and (count % 100 ==0) : print(f"seen = {count}")
            
        self.len = count
    
    def __len__(self):
        return self.len
     
    def sample(self, high, low=0, s=1):
        sample = np.random.choice(high-low, s, replace=False)
        return sample + low
    
    def __getitem__(self, idx, sample_size='auto'): 
        #for each sample:
        #sample i, j st i != j the two sequences to compare
        #sample k the sequence position of the prediction
        #compute the file positions of the 25 + 1 AA
        
        X = []
        y = []
        
        PIDs = []
        local_PIDs = []
        
        pfreqs = []
        
        pos = []
        
        precomputed_pos = []
        for i in range(-self.cont_size,self.cont_size+1):
            precomputed_pos.append(i)
        for i in range(-self.cont_size,0):
            precomputed_pos.append(i)
        for i in range(1,self.cont_size+1):
            precomputed_pos.append(i)
        
        precomputed_pos = torch.tensor(precomputed_pos).float()
        
        data_path = os.path.join(self.data_dir, self.paths[idx])
        try:
            n = self.seq_nums[idx]
            l = self.seq_lens[idx]
        except:
            print(idx)
            pass
        
        if type(sample_size) != int:
            coef = round((n**2 * l)/self.div) 
            sample_size = max(1,coef)
        
        p = l // self.col_size
        r = l % self.col_size # l = p * q + r
        sequence_line_count = p+2 if r else p+1


        for _ in range(sample_size):
            i,j = self.sample(n,s=2)

            start_i = 2 + (sequence_line_count)*i #start line of protein i
            start_j = 2 + (sequence_line_count)*j #start line of protein j
            
            seq_i = ''
            seq_j = ''
            
            PID_ij = 0
            
            l_ij = 0
            for offset in range(sequence_line_count-1):
                line_i = linecache.getline(data_path, (start_i + offset))[:-1]
                line_j = linecache.getline(data_path, (start_j + offset))[:-1]
                for aa_i, aa_j in zip(line_i,line_j):
                    if aa_i == aa_j:
                        if aa_i != '-':
                            PID_ij += 1
                            seq_i += aa_i
                            seq_j += aa_j        
                    else:
                        seq_i += aa_i
                        seq_j += aa_j
                    
                    if aa_i != '-' and aa_j != '-':
                        l_ij += 1
            
            try:
                PID_ij = PID_ij/l_ij
            except:
                PID_ij = 0
            
            align_l = len(seq_i)
            possible_k = []
            for k,(a_i,a_j) in enumerate(zip(seq_i,seq_j)):   
                if ALPHABET.get(a_i,21) < 20 and ALPHABET.get(a_j,21) < 20:
                    possible_k.append(k)
                    
            try:   
                k = np.random.choice(possible_k)
            except:
                continue
                
            pos_ij = (k + precomputed_pos)/align_l
            pos.append(pos_ij)
            
            window_i = ''
            window_j = ''
            
            for w in range(k-self.cont_size,k+self.cont_size+1):
                if w < 0 or w >= align_l:
                    window_i += 'Z'
                    window_j += 'Z'
                else:
                    window_i += seq_i[w]
                    window_j += seq_j[w]
        
            y_j = ALPHABET.get(window_j[self.cont_size], 21) # 'Z' is the default value for rare AA
            X_i = [ALPHABET.get(i, 21) for i in (window_i+window_j[:self.cont_size]+window_j[self.cont_size+1:])]       
            
            X.append(X_i)
            y.append(y_j)
            PIDs.append(PID_ij)
            local_PID_ij = sum(1 for AA1,AA2 in zip(window_i, window_j[:self.cont_size]) if AA1 == AA2 and ALPHABET.get(AA1,21) < 20) \
                         + sum(1 for AA1,AA2 in zip(reversed(window_i), reversed(window_j[self.cont_size+1:])) if AA1 == AA2 and ALPHABET.get(AA1,21) < 20)
            
            loc_comp = sum(1 for AA1,AA2 in zip(window_i, window_j[:self.cont_size]) if ALPHABET.get(AA1,21) < 20 and ALPHABET.get(AA2,21) < 20) \
                         + sum(1 for AA1,AA2 in zip(reversed(window_i), reversed(window_j[self.cont_size+1:])) if ALPHABET.get(AA1,21) < 20 and ALPHABET.get(AA2,21) < 20)
            try:
                tmp = local_PID_ij/loc_comp
            except:
                tmp = 0
                
            local_PIDs.append(tmp)
            pfreqs.append(self.aa_freqs[idx])
            
            assert y_j < 20
            assert X_i[self.cont_size] < 20
            
        linecache.clearcache()   
        X = torch.tensor(X)
        try:
            X = F.one_hot(X,22)[:,:,0:-1]
        except RuntimeError:
            pass
        if len(pos) == 0:
            pos = torch.tensor(pos)
        else:
            pos = torch.stack(pos)
        pfreqs = torch.stack(pfreqs)
        out = X.float(),torch.tensor(y).long(),torch.tensor(PIDs),torch.tensor(local_PIDs),pfreqs,pos
        return out

    

In [6]:
"""
train_dataset = MyDataset(r"data/train_data",cont_size = 6)
test_dataset = MyDataset(r"data/test_data",cont_size = 6,div=700000)
val_dataset = MyDataset(r"data/val_data",cont_size = 6,div=700000)

fname = 'data/train_dataset.pth'
savepath = Path(fname)
with savepath.open("wb") as fp:
    torch.save(train_dataset,fp)
    
fname = 'data/test_dataset.pth'
savepath = Path(fname)
with savepath.open("wb") as fp:
    torch.save(test_dataset,fp)
    
fname = 'data/val_dataset.pth'
savepath = Path(fname)
with savepath.open("wb") as fp:
    torch.save(val_dataset,fp)

"""

'\ntrain_dataset = MyDataset(r"data/train_data",cont_size = 6)\ntest_dataset = MyDataset(r"data/test_data",cont_size = 6,div=700000)\nval_dataset = MyDataset(r"data/val_data",cont_size = 6,div=700000)\n\nfname = \'data/train_dataset.pth\'\nsavepath = Path(fname)\nwith savepath.open("wb") as fp:\n    torch.save(train_dataset,fp)\n    \nfname = \'data/test_dataset.pth\'\nsavepath = Path(fname)\nwith savepath.open("wb") as fp:\n    torch.save(test_dataset,fp)\n    \nfname = \'data/val_dataset.pth\'\nsavepath = Path(fname)\nwith savepath.open("wb") as fp:\n    torch.save(val_dataset,fp)\n\n'

In [7]:
fname = 'data/train_dataset.pth'
savepath = Path(fname)
with savepath.open("rb") as fp:
    train_dataset = torch.load(fp)
    
fname = 'data/test_dataset.pth'
savepath = Path(fname)
with savepath.open("rb") as fp:
    test_dataset = torch.load(fp)
    
fname = 'data/val_dataset.pth'
savepath = Path(fname)
with savepath.open("rb") as fp:
    val_dataset = torch.load(fp)
    
print(len(train_dataset))
print(len(test_dataset))
print(len(val_dataset))

13219
2838
2826


In [ ]:
def my_collate(batch):
    data = torch.cat([item[0] for item in batch],dim=0)
    target = torch.cat([item[1] for item in batch],dim=0)
    PID = torch.cat([item[2] for item in batch],dim=0)
    lPID = torch.cat([item[3] for item in batch],dim=0)
    pfreqs = torch.cat([item[4] for item in batch],dim=0) 
    pos = torch.cat([item[5] for item in batch],dim=0)
    return data, target, PID, lPID,pfreqs, pos

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True,collate_fn=my_collate,num_workers=4)

test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True,collate_fn=my_collate,num_workers=4)

val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True,collate_fn=my_collate,num_workers=4)

In [10]:
for X, y, PID, lPID,pfreqs, pos in val_dataloader:
    print(X.shape)
    print(y.shape)
    print(PID.shape)
    print(lPID.shape)
    print(pfreqs.shape)
    print(pos.shape)
    break

torch.Size([255, 25, 21])
torch.Size([255])
torch.Size([255])
torch.Size([255])
torch.Size([255, 20])
torch.Size([255, 25])


In [11]:
class AttBlock(nn.Module):
    def __init__(self,in_features,out_features=None,num_heads=8,head_dims=24):
        super().__init__()
        out_features = out_features or in_features
        
        self.Q_w = nn.Linear(in_features,num_heads*head_dims,bias=False)
        self.K_w = nn.Linear(in_features,num_heads*head_dims,bias=False)
        self.V_w = nn.Linear(in_features,num_heads*head_dims,bias=False)
        
        self.att = nn.MultiheadAttention(num_heads*head_dims,num_heads=num_heads,batch_first=True)
        self.lin = nn.Linear(num_heads*head_dims,out_features)
        
    def forward(self,x):
        Q = self.Q_w(x)
        K = self.K_w(x)
        V = self.V_w(x)
        out,_ = self.att(Q,K,V,need_weights=False)
        out = self.lin(out)
        
        return out

In [12]:
class FeedFoward(nn.Module):
    def __init__(self,in_features,out_features=None,wide_factor=4):
        super().__init__()
        out_features = out_features or in_features
        hidden_dim = wide_factor * in_features
        
        self.lin1 = nn.Linear(in_features,hidden_dim)
        self.act1 = nn.GELU()
        self.lin2 = nn.Linear(hidden_dim,out_features)
        self.act2 = nn.GELU()
        
    def forward(self,x):
        out = self.lin1(x)
        out = self.act1(out)
        out = self.lin2(out)
        out = self.act2(out)
        
        return out

In [13]:
def drop_path(x, drop_prob: float = 0.1, training: bool = False, scale_by_keep: bool = True):
    """Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).

    This is the same as the DropConnect impl I created for EfficientNet, etc networks, however,
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ... I've opted for
    changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use
    'survival rate' as the argument.

    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = x.new_empty(shape).bernoulli_(keep_prob)
    if keep_prob > 0.0 and scale_by_keep:
        random_tensor.div_(keep_prob)
    return x * random_tensor


class DropPath(nn.Module):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """
    def __init__(self, drop_prob=None, scale_by_keep=True):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob
        self.scale_by_keep = scale_by_keep

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training, self.scale_by_keep)


In [14]:
class Block(nn.Module):
    def __init__(self,in_features,num_heads=8,head_dims=24,wide_factor=4,drop=0.1):
        super().__init__()
        
        self.att_block = AttBlock(in_features,num_heads=num_heads,head_dims=head_dims)
        self.ff = FeedFoward(in_features,wide_factor=wide_factor)
        self.drop_path = DropPath(drop)
        
        self.norm1 = nn.LayerNorm(in_features)
        self.norm2 = nn.LayerNorm(in_features)
    def forward(self,x):
        out = x + self.drop_path(self.att_block(x))
        out = self.norm1(out)
        out = out + self.drop_path(self.ff(out))
        out = self.norm2(out)
        return out

In [15]:
class Classifier_Head(nn.Module):
    def __init__(self,in_features,clf_dims,out_size,seq_len):
        super().__init__()
        in_dim = seq_len*in_features
        self.in_dim = in_dim
        
        layers = []
        for out_dim in clf_dims:
            layers.append(nn.Linear(in_dim,out_dim))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(p=0.2))
            in_dim = out_dim

        layers.append(nn.Linear(in_dim,out_size))
        
        self.clf = nn.Sequential(*layers)
        
    def forward(self,x):
        out = x.reshape((-1,self.in_dim))
        
        out = self.clf(out)
        
        return out

In [16]:
class AttNet(nn.Module):
    def __init__(self,in_features,num_heads,head_dims,wide_factors,drops,out_size=20,seq_len=28,clf_dims=[256,64],cont_size=6):
        super().__init__()
        
        blocks = []
        for n_h, h_d,w,d in zip(num_heads,head_dims,wide_factors,drops):
            blocks.append(Block(in_features,num_heads=n_h,head_dims=h_d,wide_factor=w,drop=d))
        self.feature_extractor = nn.Sequential(*blocks)
        
        self.clf = Classifier_Head(in_features,clf_dims,out_size=out_size,seq_len=seq_len)
        
        self.cont_size=cont_size
        
        sp = Path("data/freq.pth")
        with sp.open("rb") as fp:
            self.F = nn.Parameter(torch.load(fp))
            
        pid_layers = [nn.Linear(1,in_features),nn.Sigmoid()]
        self.pid_l = nn.Sequential(*pid_layers)
        
    def forward(self,x,PID,pfreqs,pos):
        X_idx = torch.argmax(x[:,self.cont_size],dim=1)
        
        X_pos = torch.cat([x,pos.unsqueeze(2)],dim=2)
        Fr = F.pad(self.F[X_idx].unsqueeze(1), pad=(0, 2), mode='constant', value=0)
        pf = F.pad(pfreqs.unsqueeze(1),pad =(0,2), mode='constant', value=0)
        
        pid = self.pid_l(PID.unsqueeze(1)).unsqueeze(1)
        
        X_input = torch.cat([X_pos,Fr,pf,pid],dim=1)
                
        features = self.feature_extractor(X_input)
        out = self.clf(features)
        return out

In [17]:
class State:
    def __init__(self,model,optim,scheduler):
        self.model = model
        self.optim = optim
        self.scheduler = scheduler
        self.epoch = 0

In [18]:
def eval_model(fname,train,test,val):
    savepath = Path(fname)
    with savepath.open("rb") as fp:
        state = torch.load(fp)
        
    state.model.eval()
    with torch.no_grad():
        print('EVALUATING ON TRAIN DATA : ')
        eval_losses = []
        for X,y, PID, lPID,pfreqs,pos in train:
            X = X.to(device)
            y = y.to(device)
            PID = PID.to(device)
            pos = pos.to(device)
            pfreqs = pfreqs.to(device)

            y_hat = state.model(X,PID,pfreqs,pos)
            y_hat = F.softmax(y_hat,dim=1)

            y = F.one_hot(y, 20)
            eval_l = (y_hat-y)**2
            eval_losses.append(torch.sum(eval_l,dim=1).mean().detach().cpu())

        score_train = torch.mean(torch.stack(eval_losses)).detach().cpu().item()
        print(f"{score_train = }\n")

        print('EVALUATING ON TEST DATA : ')
        eval_losses = []
        for X,y, PID, lPID,pfreqs, pos in test:
            X = X.to(device)
            y = y.to(device)
            PID = PID.to(device)
            pos = pos.to(device)
            pfreqs = pfreqs.to(device)

            y_hat = state.model(X,PID,pfreqs,pos)
            y_hat = F.softmax(y_hat,dim=1)

            y = F.one_hot(y, 20)
            eval_l = (y_hat-y)**2
            eval_losses.append(torch.sum(eval_l,dim=1).mean().detach().cpu())

        score_test = torch.mean(torch.stack(eval_losses)).detach().cpu().item()
        print(f"{score_test = }\n")

        print('EVALUATING ON VAL DATA : ')
        eval_losses = []
        for X,y, PID, lPID,pfreqs, pos in val:
            X = X.to(device)
            y = y.to(device)
            PID = PID.to(device)
            lPID = lPID.to(device)
            pos = pos.to(device)
            pfreqs = pfreqs.to(device)

            y_hat = state.model(X,PID,pfreqs,pos)
            y_hat = F.softmax(y_hat,dim=1)

            y = F.one_hot(y, 20)
            eval_l = (y_hat-y)**2
            eval_losses.append(torch.sum(eval_l,dim=1).mean().detach().cpu())

        score_val = torch.mean(torch.stack(eval_losses)).detach().cpu().item()
        print(f"{score_val = }\n")
    
    return score_train, score_test, score_val

In [19]:
def main(train_loader,val_loader,epochs=101,fname="models/state.pth",fnameb=None,state=None,last_epoch_sched=495):
    
    #to get the best model
    best = float('inf')
    
    #getting the acceleration device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    #loading from previous checkpoint
    if fnameb is None:
        fnameb = fname[:-4] + '_best' +fname[-4:]
        
    savepath = Path(fname)
    if savepath.is_file():
        with savepath.open("rb") as fp:
            state = torch.load(fp)
    else:
        if state is None:
            model = AttNet(22,[8,8,8],[24,24,24],[4,4,4],[0.1,0.1,0.1])
            model = model.to(device)
            optim = torch.optim.AdamW(model.parameters(),lr = 0.0001)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim,16,2)
            state = State(model,optim,scheduler)
    
    
    Loss = nn.CrossEntropyLoss()
    EvalLoss = nn.MSELoss()
    
    #for logs
    List_Loss = []
    Eval_Loss = []
    for epoch in range(state.epoch, epochs):
        batch_losses = []
        state.model.train()
        for X,y, PID, lPID,pfreqs,pos in train_loader:
            X = X.to(device)
            y = y.to(device)
            PID = PID.to(device)
            pos = pos.to(device)
            pfreqs = pfreqs.to(device)
               
            state.optim.zero_grad()
            y_hat = state.model(X,PID,pfreqs,pos)
            
            l = Loss(y_hat,y)
            l.backward()
            state.optim.step()
            
            
            
            batch_losses.append(l.detach().cpu())
        List_Loss.append(torch.mean(torch.stack(batch_losses)).detach().cpu())
        state.epoch = epoch + 1
        if epoch < last_epoch_sched:
            state.scheduler.step()
        
        savepath = Path(fname)
        with savepath.open("wb") as fp:
            torch.save(state,fp)
        
        with torch.no_grad():
            eval_losses = [] 
            state.model.eval()
            for X,y, PID, lPID,pfreqs, pos in val_loader:

                X = X.to(device)
                y = y.to(device)
                PID = PID.to(device)
                pos = pos.to(device)
                pfreqs = pfreqs.to(device)

                y_hat = state.model(X,PID,pfreqs,pos)
                y_hat = F.softmax(y_hat,dim=1)

                y = F.one_hot(y, 20)
                eval_l = (y_hat-y)**2
                eval_losses.append(torch.sum(eval_l,dim=1).mean().detach().cpu())


            score = torch.mean(torch.stack(eval_losses)).detach().cpu()
            Eval_Loss.append(score)
        
        if score < best :
            best = score
            savepath = Path(fnameb)
            with savepath.open("wb") as fp:
                torch.save(state,fp)
        
        print(f"epoch n°{epoch} : train_loss = {List_Loss[-1]}, val_loss = {Eval_Loss[-1]}") 


        
    return List_Loss,Eval_Loss,state

In [ ]:
model = AttNet(22,[8,8,8,8,8,8],[24,24,24,24,24,24],[4,4,4,4,4,4],[0.1,0.1,0.1,0.1,0.1,0.1])
model = model.to(device)
optim = torch.optim.AdamW(model.parameters(),lr = 0.0003)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim,16,2,eta_min=3e-6)
state = State(model,optim,scheduler)

fname = "models/stateATT6_freq.pth" 
start = time.time()
Train,Eval,_ = main(train_dataloader, val_dataloader,fname=fname,epochs=2032,state=state)
stop = time.time()
print(stop-start)

epoch n°0 : train_loss = 2.6879584789276123, val_loss = 0.8432316184043884
epoch n°1 : train_loss = 2.426236867904663, val_loss = 0.8241277933120728
epoch n°2 : train_loss = 2.376164674758911, val_loss = 0.8169472813606262
epoch n°3 : train_loss = 2.3694581985473633, val_loss = 0.8200643062591553
epoch n°4 : train_loss = 2.3478612899780273, val_loss = 0.8191378116607666
epoch n°5 : train_loss = 2.355975866317749, val_loss = 0.8105881810188293
epoch n°6 : train_loss = 2.3170580863952637, val_loss = 0.8130093216896057
epoch n°7 : train_loss = 2.3188490867614746, val_loss = 0.8171864748001099
epoch n°8 : train_loss = 2.3135881423950195, val_loss = 0.8109681010246277
epoch n°9 : train_loss = 2.320058822631836, val_loss = 0.81395024061203
epoch n°10 : train_loss = 2.317624807357788, val_loss = 0.8152602314949036
epoch n°11 : train_loss = 2.3079471588134766, val_loss = 0.8075902462005615
epoch n°12 : train_loss = 2.311197280883789, val_loss = 0.8085429072380066
epoch n°13 : train_loss = 2.30

In [ ]:
plt.plot(np.arange(len(Train)),Train)

In [ ]:
plt.plot(np.arange(len(Eval)),Eval)

In [ ]:
fname = "models/stateATT6_freq.pth" 
SCORES = eval_model(fname,train_dataloader,test_dataloader,val_dataloader)

In [ ]:
fname = "models/stateATT6_freq_best.pth" 
SCORES = eval_model(fname,train_dataloader,test_dataloader,val_dataloader)

In [ ]:
def get_params(input_size,N,head,head_dim,wide_factor,drop_prob):
    return input_size, [head for _ in range(N)], [head_dim for _ in range(N)], [wide_factor for _ in range(N)], [drop_prob for _ in range(N)], 